In [1]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0
T7_DNA.lc =0

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.1
Repressor_DNA.lc = 0.1


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [3]:
#Positive control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]

result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(97,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).pc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.result)

In [4]:
#Negative control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(97,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).nc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(R.nc_result)

In [5]:
stored_data = []
for conc in np.logspace(-4,1, 100, endpoint=True):
    #experiment 
    T7_DNA.ic = conc
    T7_DNA.lc = conc
    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic

    #stage 1 kick start
    result_all=[molecules_0] #initial condition


    #DR = 0.2

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



    #wash out
    dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]

    result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



    result_all = np.resize(result_all,(97,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
    #plot_result(GFP.result)
    #print((GFP).result[60]/GFP.pc_result[60])
    stored_data.append((GFP).result[60])



In [6]:
conc = np.logspace(-4,1, 100, endpoint=True)

p = figure(width=800, height=400,x_axis_type = "log")
p.dot(conc,stored_data,size=20)
show(p)

In [7]:
print(GFP.pc_result)

[ 0.         10.76776562 36.22342082 52.51976303 54.43678623 55.79918023
 56.76225426 57.44033149 57.91634883 58.24980859 58.48304892 58.64601413
 58.75979128 58.83918429 58.89456343 58.93318196 58.96010755 58.97887819
 58.99196259 59.00108273 59.0074394  59.01186983 59.01495764 59.01710969
 59.01860954 59.01965483 59.02038333 59.02089104 59.02124488 59.02149148
 59.02166334 59.02178311 59.02186659 59.02192476 59.0219653  59.02199356
 59.02201325 59.02202697 59.02203654 59.0220432  59.02204785 59.02205109
 59.02205334 59.02205492 59.02205601 59.02205678 59.02205731 59.02205768
 59.02205794 59.02205812 59.02205824 59.02205833 59.02205839 59.02205843
 59.02205846 59.02205849 59.0220585  59.02205851 59.02205852 59.02205852
 59.02205852 59.02205853 59.02205853 59.02205853 59.02205853 59.02205853
 59.02205853 59.02205853 59.02205853 59.02205853 59.02205853 59.02205853
 59.02205853 59.02205853 59.02205853 59.02205853 59.02205853 59.02205853
 59.02205853 59.02205853 59.02205853 59.02205853 59